In [2]:
import sys
import os
from sage.all import *
import random
from matplotlib.colors import CSS4_COLORS
from itertools import product
import sage.matrix.matrix_integer_dense_hnf as hnf
from sage.all import FractionField, vector, Matrix
from IPython.display import display, Math, Latex

import sage.typeset.character_art as character_art
from sage.repl.rich_output import get_display_manager
dm = get_display_manager()
dm.preferences.text = 'ascii_art'
import sage.typeset.character_art as character_art
character_art.MAX_WIDTH = 100

sage_folder = os.path.abspath("../sage")

if sage_folder not in sys.path:
    sys.path.append(sage_folder)

from loader import load_from_dir

# on import toutes les fonctions sages faites maison dont on a besoin
load_from_dir("../sage/nint.sage", globals())
load_from_dir("../sage/nmatrix.sage", globals())
load_from_dir("../sage/row_degree.sage", globals())
load_from_dir("../sage/shifted_row_degree.sage", globals())
load_from_dir("../sage/is_in_lattice.sage", globals())
load_from_dir("../sage/is_lattice_basis.sage", globals())
load_from_dir("../sage/random/random_full_rank_matrix.sage", globals())
load_from_dir("../sage/random/random_matrix_of_rank.sage", globals())
load_from_dir("../sage/random/random_GLZ_matrix.sage", globals())
load_from_dir("../sage/gram_schmidt.sage", globals())

### 1. Arrondi à l'entier le plus proche (*Nearest Integer*)

**Définition.**
Soit $x \in \mathbb{R}$. On définit l'entier le plus proche de $x$, noté $\lceil x\rfloor$, par la formule :

$$\lceil x\rfloor = \lfloor x + 1/2 \rfloor$$

> **Note sur la convention ($0.5$) :**
>
> Cette définition implique un arrondi vers l'infini positif pour les demi-entiers.
>
> *Exemple :* $0.5 \longrightarrow 1$ (et non $0$).

**Implémentation SageMath :**
Cette fonction est disponible sous le nom `nint`.

In [3]:
tests = [-4.23, 0.5, 1.499]

for t in tests:
    display(Math(rf"{t:.3f} \longrightarrow {nint(t)}"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### 1.2. Matrice d'entiers la plus proche (*Nearest Matrix*)

**Définition.**
Soit $A \in \mathcal{M}_{n,m}(\mathbb{R})$ une matrice à coefficients réels.
On définit la matrice arrondie, notée $\lceil A \rfloor$, en appliquant la fonction *nearest integer* à chaque coefficient :

$$(\lceil A \rfloor)_{i,j} = \lceil A_{i,j} \rfloor$$

**Implémentation :**
La fonction `nmatrix(M)` applique `nint` élément par élément.

In [4]:
M_test = matrix([
    [-4.23, 0.5,  2.1],
    [ 3.99, -0.5, 10.0]
])

M_round = nmatrix(M_test)

display(Math(r"\text{Exemple de transformation :}"))

display(Math(
    rf"{latex(M_test)} \longrightarrow {latex(M_round)}"
))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### 2. Degré en ligne et Degré décalé

**Définition 1 (Degré en ligne).**
* Pour un vecteur ligne $\mathbf{m} = (m_1, \dots ,m_n) \in \mathbb{K}[x]^{1 \times n}$, on définit son **degré en ligne** par :
  $$\text{rdeg}(\mathbf{m}) = \max_{1 \leq j \leq n} \deg(m_j) \in \mathbb{Z}$$

* Pour une matrice $M$ composée de lignes $\mathbf{M_1}, \dots, \mathbf{M_r}$, on définit son **degré en ligne** par le vecteur :
  $$\text{rdeg}(M) = (\text{rdeg}(\mathbf{M_i}))_{1 \leq i \leq r} \in \mathbb{Z}^r$$

---

**Définition 2 (Degré décalé).**
Soit $\vec{\mathbf{s}} =(s_1, \dots, s_n) \in \mathbb{Z}^n$ un vecteur d'entiers appelé **vecteur de décalage**.

* Pour un vecteur ligne $\mathbf{m} = (m_1, \dots ,m_n)$, on définit son **degré en ligne $\vec{\mathbf{s}}$-décalé** par :
  $$\text{rdeg}_{\vec{\mathbf{s}}}(\mathbf{m}) = \max_{1 \leq j \leq n} (\deg(m_j) + s_j)$$

* Pour une matrice $M$, le **degré en ligne décalé** est le vecteur formé par les degrés décalés de ses lignes :
  $$\text{rdeg}_{\vec{\mathbf{s}}}(M) = \big( \text{rdeg}_{\vec{\mathbf{s}}}(\mathbf{M_i}) \big)_{1 \leq i \leq r} \in \mathbb{Z}^r$$

In [5]:
from IPython.display import display, Math

# 1. Préparation de l'anneau et des outils
R.<x> = ZZ[]
# Rappel : assurez-vous d'avoir chargé vos fonctions row_degree et shifted_row_degree avant !

# 2. Exemple de Matrice M (2 lignes, 2 colonnes)
# Ligne 1 : (x^2 + 1,  x)      -> degs: (2, 1)
# Ligne 2 : (5,        x^3-x)  -> degs: (0, 3)
M = matrix([
    [x^2 + 1,  x],
    [5,        x^3 - x]
])

# 3. Définition du vecteur de décalage s
# On ajoute 0 à la col 1, et 2 à la col 2
s = [0, 2]

# 4. Calculs
res_rdeg = row_degree(M)
res_shifted = shifted_row_degree(M, s)

# 5. Affichage "Pro" avec display Math
display(Math(r"\textbf{Exemple sur } \mathbb{Z}[x] :"))

# Affichage du degré simple
display(Math(
    rf"\text{{rdeg}}\left( {latex(M)} \right) = {latex(res_rdeg)}"
))

# Affichage du vecteur de décalage
display(Math(rf"\text{{Avec le vecteur de décalage }} \vec{{\mathbf{{s}}}} = {s} :"))

# Affichage du degré décalé
# Détail pour vérification visuelle :
# Ligne 1 décalée : max(2+0, 1+2) = max(2, 3) = 3
# Ligne 2 décalée : max(0+0, 3+2) = max(0, 5) = 5
display(Math(
    rf"\text{{rdeg}}_{{\vec{{\mathbf{{s}}}}}}(M) = {latex(res_shifted)}"
))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Test d'appartenance (Membership Test)

**Problème :** Soit un réseau $\mathcal{L}$ engendré par les lignes d'une matrice $L$, et un vecteur $v$. On veut déterminer si $v \in \mathcal{L}$.

**Méthode via HNF :**
On utilise la propriété d'unicité de la Forme Normale d'Hermite.
Le vecteur $v$ appartient au réseau si et seulement si l'ajout de ce vecteur à la famille génératrice ne modifie pas le réseau engendré.

$$v \in \mathcal{L}(L) \iff \text{HNF}(L) = \text{HNF}\left(\begin{pmatrix} L \\ v \end{pmatrix}\right)$$

**Implémentation :**
On compare la forme normale d'Hermite de $L$ avec celle de la matrice augmentée (L empilée avec v).

In [ ]:
# 1. Création d'un réseau L pour l'exemple
# L engendré par (3, 1) et (1, 2)
L = matrix(ZZ, [
    [3, 1],
    [1, 2]
])

display(Math(r"\textbf{1. Test d'appartenance (Membership)}"))
display(Math(rf"\text{{Réseau }} L = {latex(L)}"))

# --- CAS 1 : Vecteur DANS le réseau ---
# v_in = 1*(3,1) - 1*(1,2) = (2, -1). C'est une combinaison entière.
v_in = vector(ZZ, [2, -1])
res_in = is_in_lattice(v_in, L)

display(Math(
    rf"\vec{{v}}_{{in}} = {latex(v_in)} \implies "
    rf"\text{{is\_in\_lattice? }} \mathbf{{{res_in}}} "
    r"\quad (\because \vec{v}_{in} = L_1 - L_2)"
))

# --- CAS 2 : Vecteur HORS du réseau ---
# v_out = (1, 0). Impossible à atteindre avec des entiers.
v_out = vector(ZZ, [1, 0])
res_out = is_in_lattice(v_out, L)

display(Math(
    rf"\vec{{v}}_{{out}} = {latex(v_out)} \implies "
    rf"\text{{is\_in\_lattice? }} \mathbf{{{res_out}}}"
))

print("-" * 20)

# --- CAS 3 : Isomorphisme / Même Réseau ---
display(Math(r"\textbf{2. Test d'égalité de réseaux (Isomorphism)}"))

# On génère une autre base du MÊME réseau via LLL
L_reduced = L.LLL() 

res_iso = is_lattice_basis(L_reduced, L)

display(Math(rf"\text{{Base 1 (Originale) }} L = {latex(L)}"))
display(Math(rf"\text{{Base 2 (LLL) }} L_{{LLL}} = {latex(L_reduced)}"))

display(Math(
    rf"\text{{is\_lattice\_basis}}(L, L_{{LLL}}) \longrightarrow \mathbf{{{res_iso}}}"
))

### Test de Base de Réseau (Basis Check)

**Problème :** Soit un réseau $\mathcal{L}$ défini par une base $L$. On veut savoir si une autre matrice $B$ est aussi une base de ce même réseau.

**Méthode par le déterminant :**
Une matrice $B$ est une base de $\mathcal{L}(L)$ si et seulement si deux conditions sont réunies :
1.  **Inclusion :** Tous les vecteurs lignes de $B$ appartiennent à $\mathcal{L}(L)$ (c'est-à-dire $\mathcal{L}(B) \subseteq \mathcal{L}(L)$).
2.  **Volume :** Les déterminants sont égaux en valeur absolue (les réseaux ont le même covolume).

$$\mathcal{L}(B) = \mathcal{L}(L) \iff \left( \forall i, B_i \in \mathcal{L}(L) \right) \text{ et } |\det(B)| = |\det(L)|$$

**Implémentation :**
On vérifie l'inclusion avec `is_in_lattice` pour chaque ligne, puis on compare les déterminants absolus.

### Test d'Orthogonalité

**Définition.**
Une famille de vecteurs (les lignes d'une matrice $B$) est dite **orthogonale** si les vecteurs sont deux à deux orthogonaux pour le produit scalaire canonique :
$$\forall i \neq j, \quad \langle b_i, b_j \rangle = 0$$

**Caractérisation matricielle :**
Cela équivaut à dire que la matrice de Gram $G = B \times B^T$ est une matrice **diagonale**.
Les coefficients sur la diagonale correspondent alors aux normes au carré des vecteurs ($\|b_i\|^2$).